In [3]:
!pip install rdflib sentence-transformers chromadb

from rdflib import Graph, Namespace, RDF, URIRef
from sentence_transformers import SentenceTransformer
import chromadb

# Create RDF Graph
g = Graph()

# Define namespace
EX = Namespace("http://example.org/")

# Define entities
alice = EX.Alice
bob = EX.Bob
acme = EX.AcmeCorp

# Define predicates
knows = EX.knows
worksAt = EX.worksAt

# Add triples
g.add((alice, RDF.type, EX.Person))
g.add((bob, RDF.type, EX.Person))
g.add((alice, knows, bob))
g.add((bob, worksAt, acme))

sentences = []

for s, p, o in g:
    s_name = s.split("/")[-1]
    p_name = p.split("/")[-1]
    o_name = o.split("/")[-1]

    sentence = f"{s_name} {p_name} {o_name}"
    sentences.append(sentence)

sentences

['Alice type Person',
 'Alice knows Bob',
 'Bob worksAt AcmeCorp',
 'Bob type Person']

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(sentences)

client = chromadb.Client()

collection = client.get_or_create_collection(name="kg_embeddings")

for i, sentence in enumerate(sentences):
    collection.add(
        documents=[sentence],
        embeddings=[embeddings[i].tolist()],
        ids=[str(i)]
    )

query = input("Enter your query: ")

query_embedding = model.encode([query])

results = collection.query(
    query_embeddings=query_embedding.tolist(),
    n_results=1
)
print("\n🔍 Answer:")
print(results["documents"][0][0])





Enter your query: who is Alice know

🔍 Answer:
Alice knows Bob
